In [1]:
import statistics

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
train = pd.read_csv('data/train.csv')

rubric_mapping = dict(
    zip(
        pd.unique(train.rubrics),
        np.arange(0, len(pd.unique(train.rubrics)))
    )
)
city_mapping = {'msk': 0, 'spb': 1}

train.rubrics = train.rubrics.apply(lambda it: rubric_mapping[it])
train.user_city = train.user_city.apply(lambda it: city_mapping[it])
train.org_city = train.org_city.apply(lambda it: city_mapping[it])

mean = train.average_bill.mean()
train.average_bill.fillna(mean, inplace=True)
train.fillna(method='ffill', inplace=True)

# All columns except ration_org, user_id, org_id
# Eq to train.drop(['user_id', 'org_id', 'user_city'], axis=1, inplace=True)
# columns = ['rating', 'cafe', 'toilet_for_disabled', 'parking_disabled', 'payment_by_credit_card', 'automatic_door', 'wheelchair_access', 'car_park', 'craft_beer', 'gift_certificate', 'food_delivery', 'sports_broadcasts', 'privilege_tickets', 'projector', 'wi_fi', 'food_product_delivery', 'karaoke', 'free_delivery', 'farm_products', 'self_service_kiosks', 'organic_food', 'minimum_order', 'table_games', 'promotions', 'takeaway', 'elevator_wheelchair_accessible', 'average_bill', 'music', 'vip_zone', 'delivery', 'face_control', 'dancefloor', 'show_program', 'rubrics', 'food_court1', 'dress_code', 'coffee_to_go', 'chillout', 'air_conditioning', 'elevator', 'internet access', 'strip', 'wheelchair_accessible', 'handmade_goods', 'pickup', 'has_bar', 'hall_capacity', 'around_the_clock_work1', 'call_button', 'summer_terrace', 'ramp', 'teahouse', 'pets', 'coffee_shop', 'kalyan', 'billiards', 'org_city', 'has_restaurant', 'cakes_for_different_occasions', 'accepted_credit_cards', 'online_takeaway', 'bread_from_tandoor', 'business_lunch', 'user_city', 'vinotheque', 'special_menu', 'closed_for_quarantine', 'breakfast']
columns = ['rating']
    # , 'cafe', 'payment_by_credit_card', 'car_park', 'wi_fi', 'music', 'rubrics']
# , 'food_court1', 'dress_code', 'coffee_to_go', 'chillout', 'air_conditioning', 'elevator', 'internet access', 'strip', 'wheelchair_accessible', 'handmade_goods', 'pickup', 'has_bar', 'hall_capacity', 'around_the_clock_work1', 'call_button', 'summer_terrace', 'ramp', 'teahouse', 'pets', 'coffee_shop', 'kalyan', 'billiards', 'org_city', 'has_restaurant', 'cakes_for_different_occasions', 'accepted_credit_cards', 'online_takeaway', 'bread_from_tandoor', 'business_lunch', 'user_city', 'vinotheque', 'special_menu', 'closed_for_quarantine', 'breakfast']

x = train[columns]

y = np.array(train.rating_org)

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
model = KNeighborsRegressor(n_neighbors=3000, weights='uniform', p=1)

model.fit(x, y)
# print(model.score(x_test, y_test))

KNeighborsRegressor(n_neighbors=3000, p=1)

In [3]:
# m = dict(zip(train.org_id, train.rating_org))

# Forecast
test = pd.read_csv('data/test_x.csv')

mean = test.average_bill.mean()
test.average_bill.fillna(mean, inplace=True)
test.fillna(method='ffill', inplace=True)

test.rubrics = test.rubrics.apply(lambda it: rubric_mapping[it])
test.user_city = test.user_city.apply(lambda it: city_mapping[it])
test.org_city = test.org_city.apply(lambda it: city_mapping[it])
x = test[columns]
# x = StandardScaler().fit_transform(x)

predicted = model.predict(x)
# for i in range(len(predicted)):
#     if test.loc(i)['org_id'] in m:
#         predicted[i] = m[test.loc(i).org_id]
res = pd.DataFrame(data=predicted, columns=['rating_org'])
res.index.name = 'id'

res.to_csv('res.csv')

In [4]:
for i in range(len(predicted)):
    org_id = test.loc[[i]]['org_id'].values[0]
    if org_id in m:
        predicted[i] = m[org_id]
res = pd.DataFrame(data=predicted, columns=['rating_org'])
res.index.name = 'id'

res.to_csv('res1.csv')

NameError: name 'm' is not defined

In [ ]:
for i in range(5, 151, 10):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
    model = KNeighborsRegressor(n_neighbors=i, weights='distance', p=1)
    model.fit(x_train, y_train)
    print(i, model.score(x_test, y_test))

In [ ]:
import statistics

cor = []
for (columnName, columnData) in train.items():
    coef = statistics.correlation(columnData, train.rating_org)
    cor.append((abs(coef), columnName))
cor.sort(reverse=True)
print(*cor, sep='\n')
sortedColumns = list(map(lambda it: it[1], cor))[1:]
sortedColumns.remove('org_id')
sortedColumns.remove('user_id')
print(sortedColumns)

In [ ]:
train = pd.read_csv('data/train.csv')

rubric_mapping = dict(
    zip(
        pd.unique(train.rubrics),
        np.arange(0, len(pd.unique(train.rubrics)))
    )
)
train.rubrics = train.rubrics.apply(lambda it: rubric_mapping[it])

city_mapping = {'msk': 0, 'spb': 1}
train.user_city = train.user_city.apply(lambda it: city_mapping[it])
train.org_city = train.org_city.apply(lambda it: city_mapping[it])

mean = train.average_bill.mean()
train.average_bill.fillna(mean, inplace=True)



y = np.array(train.rating_org)

model = KNeighborsRegressor(n_neighbors=20, weights='uniform', p=2)

train[train.columns] = StandardScaler().fit_transform(train[train.columns])

for i in range(25, len(sortedColumns)):
    columns = sortedColumns[:i]
    x = train[columns]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
    model.fit(x_train, y_train)
    print(i, model.score(x_test, y_test), columns[-1])

## Как это работает?

Выбираем точку u и подбираем для неё категорию тета.

![title](img.png)

In [ ]:
train = pd.read_csv('data/train.csv')
train.drop(['user_id', 'org_id'], axis=1, inplace=True)
train.fillna(method='ffill', inplace=True)

ohe = OneHotEncoder()
transformedRubrics = ohe.fit_transform(train[['rubrics']])
rubricsCategories = ohe.categories_[0]
train[rubricsCategories] = transformedRubrics.toarray()

ohe = OneHotEncoder()
transformedOrgCity = ohe.fit_transform(train[['org_city']])
orgCityCategories = ohe.categories_[0]
train[orgCityCategories] = transformedOrgCity.toarray()

columns = ['rating', 'average_bill', 'payment_by_credit_card', 'wi_fi', 'cafe', 'has_bar', 'internet access', 'music', 'delivery', 'kalyan']
columns += rubricsCategories.tolist()
columns += orgCityCategories.tolist()
x = train[columns]

y = np.array(train.rating_org)

model = KNeighborsRegressor(n_neighbors=100, weights='uniform', p=2)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
model.fit(x_train, y_train)

print(model.score(x_test, y_test))


test = pd.read_csv('data/test_x.csv')
test.drop(['user_id', 'org_id'], axis=1, inplace=True)
test.fillna(method='ffill', inplace=True)

ohe = OneHotEncoder()
transformedRubrics = ohe.fit_transform(test[['rubrics']])
rubricsCategories = ohe.categories_[0]
test[rubricsCategories] = transformedRubrics.toarray()

ohe = OneHotEncoder()
transformedOrgCity = ohe.fit_transform(test[['org_city']])
orgCityCategories = ohe.categories_[0]
test[orgCityCategories] = transformedOrgCity.toarray()

test = test[columns]

predicted = model.predict(test)
res = pd.DataFrame(data=predicted, columns=['rating_org'])

res.index.name = 'id'
res.to_csv('res.csv')